In [8]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F
 
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
from torch.nn.common_types import _size_2_t
from torch.nn.modules.utils import _pair
from collections import OrderedDict, abc
from itertools import repeat

from typing import Type, Callable, List

# Inference and learning final project

## Introduction


## Imports

In [9]:
class D_Conv(nn.Module):
    ### only works with square image, kernel size and stride
    def __init__(self, base_channel_size: int, loc : Callable[[int,int,int,int,int],tuple[int,List[nn.Module]]], image_size :int ) -> None:
        super().__init__()
        
        alpha = base_channel_size
        channel_size :int = 3
        im_size : int = image_size
        
        modules : List[nn.Module] = []
        size_conv = [
            (alpha,1),
            (2*alpha,1), #remove stride for now
            (2*alpha,1),
            (4*alpha,1), #here too
            (4*alpha,1),
            (8*alpha,1), # here too
            (8*alpha,1),
            (16*alpha,1)] # here too
        
        for i,val in enumerate(size_conv):
            size_out, stride = val
            im_size,new_module = loc(channel_size,size_out,3,stride,im_size)
            modules.extend(new_module)
            channel_size = size_out
            
        self.conv = nn.Sequential(*modules)
        self.fc = nn.Sequential(
            nn.Linear(channel_size*im_size**2, 64*alpha),
            #nn.BatchNorm1d(64*alpha), no this either ?
            nn.ReLU(),
            nn.Linear(64*alpha, 10),
        )

    def forward(self, x):
        x = self.conv(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x


class S_Conv(nn.Module):
    def __init__(self, base_channel_size: int, loc : Callable[[int,int,int,int,int],tuple[int,List[nn.Module]]], image_size :int) -> None:
        super().__init__()
        
        alpha = base_channel_size
        channel_size :int = 3
        im_size : int = image_size
        
        im_size, modules = loc(channel_size,alpha,9,2,im_size)
        channel_size = alpha
        
        self.conv =  nn.Sequential(*modules)
        
        self.fc = nn.Sequential(
            nn.Linear(channel_size*im_size**2, 24*alpha),
            #nn.BatchNorm2d(24*alpha), Here too ?
            nn.ReLU(),
            nn.Linear(24*alpha, 10),
        )

    def forward(self, x):
        x = self.conv(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x

In [10]:
# out of memory 
class LocalLinear_custom(nn.Module):
    def __init__(self, in_channels: int, out_channels: int, kernel_size: _size_2_t, stride: _size_2_t, image_size: _size_2_t):
        super(LocalLinear_custom, self).__init__()
        self.kernel_size = _pair(kernel_size)
        image_size = _pair(image_size)
        self.stride = _pair(stride)
        self.output_size = tuple((image_size[i]-self.kernel_size[i])//self.stride[i]+1 for i in range(2))
        self.weight = nn.Parameter(torch.randn(out_channels, in_channels, *self.output_size, *self.kernel_size))
        self.bias = nn.Parameter(torch.randn(out_channels,*self.output_size))

    def forward(self, x:torch.Tensor):
        x = x.unfold(2, self.kernel_size[0], self.stride[0]).unfold(3, self.kernel_size[1], self.stride[1])
        x = x.unsqueeze(1)
        
        x = torch.matmul(x,self.weight)
        x = x.sum([-1,-2,-5]) + self.bias
        return x

class flatten_custom(nn.Module):
    def __init__(self):
        super().__init__()
    def forward(self, x):
        return torch.flatten(x, 1)


In [11]:
def funConv2d(size_in: int,size_out: int, kernel_size: int, stride: int, image_size: int ) -> tuple[int,List[nn.Module]]:
    modules = [ 
    nn.Conv2d(size_in,size_out,kernel_size,stride = stride),
    nn.BatchNorm2d(size_out),
    nn.ReLU()]
    im_out = (image_size-kernel_size)//stride+1
    return im_out,modules

def funLocalLinear(size_in: int,size_out: int, kernel_size: int, stride: int, image_size: int ) -> tuple[int,List[nn.Module]]:
    modules = [ 
    LocalLinear_custom(size_in,size_out,kernel_size,stride,image_size),
    nn.BatchNorm2d(size_out),
    nn.ReLU()]
    im_out = (image_size-kernel_size)//stride+1
    return im_out,modules

def funFullyConnected(size_in: int,size_out: int, kernel_size: int, stride: int, image_size: int ) -> tuple[int,List[nn.Module]]:
    modules = []
    if(image_size != 1):
        modules.append(flatten_custom())
    modules.extend([ 
    nn.Linear(size_in*image_size**2,size_out),
    nn.ReLU()])
    im_out = 1
    return im_out,modules

In [12]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 512

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,download=True, transform=transform)
trainloader = DataLoader(trainset, batch_size=batch_size,shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [13]:

import torch.optim as optim


networks = [(D_Conv,"D_Conv"),(S_Conv,"S_Conv")]
convolutions = [(funConv2d,"Conv2d"),(funFullyConnected,"FullyConnected")] #(funLocalLinear,"LocalLinear"),

nets = [(ni+"_"+nj,i,j) for i,ni in networks for j,nj in convolutions ]

#net = D_Conv(10,funFullyConnected,32).to(DEVICE)
#criterion = nn.CrossEntropyLoss()
#optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
#net

In [16]:
for name,net_type,call in nets:
    net = net_type(10,call,32).to(DEVICE)
    print(name)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
    
    for epoch in range(400):  # loop over the dataset multiple times
        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data[0].to(DEVICE), data[1].to(DEVICE)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
        
        print('*', end='')
        if(epoch % 25 == 24):
            print(f" {epoch+1}")

    print('\nFinished Training')
    PATH = f"networks/{name}.pth"
    torch.save(net.state_dict(), PATH)

D_Conv_Conv2d
************************* 25
************************* 50
************************* 75
************************* 100
************************* 125
************************* 150
************************* 175
************************* 200
************************* 225
************************* 250
************************* 275
************************* 300
************************* 325
************************* 350
************************* 375
************************* 400

Finished Training
D_Conv_FullyConnected
************************* 25
************************* 50
************************* 75
************************* 100
************************* 125
************************* 150
************************* 175
************************* 200
************************* 225
************************* 250
************************* 275
************************* 300
************************* 325
************************* 350
************************* 375
*********************

In [15]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(DEVICE), data[1].to(DEVICE)
        # calculate outputs by running images through the network
        outputs = net(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')

Accuracy of the network on the 10000 test images: 36 %
